In [126]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC

In [103]:
df = pd.read_csv('data/training_dataset_2018-07-05.csv', low_memory=False)

In [104]:
df = pd.get_dummies(df, columns=[
        "locale_browser_language",
        "plan_id",
        "simplified_plan_id",
        "timezone",
        "twofactor_type"
])

In [105]:
df['trialed'] = df['trial_start_at'].notna() 

In [106]:
df = df.drop([
    'customer_id',
    'created_at',
    'canceled_at',
    'trial_start_at',
    'trial_end_at',
    'user_id',
    'user_signup_date'
], axis=1)

In [107]:
df['churned_in_next_two_months'] = df['churned_in_next_two_months'].map({'t': True, 'f': False})
df['has_verified_email'] = df['has_verified_email'].map({'t': True, 'f': False})
df['has_notifications_enabled'] = df['has_notifications_enabled'].map({'t': True, 'f': False})
df['is_approved'] = df['is_approved'].map({'t': True, 'f': False})
df['is_suspended'] = df['is_suspended'].map({'t': True, 'f': False})

In [108]:
df = df.fillna(0)

In [109]:
X = df.drop(['subscription_id', 'churned_in_next_two_months'], axis=1)

In [110]:
Y = df['churned_in_next_two_months']

### Validation

In [111]:
val_df = pd.read_csv('data/validation_dataset_2018-07-05.csv', low_memory=False)

In [112]:
val_df = pd.get_dummies(val_df, columns=[
        "locale_browser_language",
        "plan_id",
        "simplified_plan_id",
        "timezone",
        "twofactor_type"
])

val_df['trialed'] = val_df['trial_start_at'].notna() 
val_df = val_df.drop([
    'customer_id',
    'created_at',
    'canceled_at',
    'trial_start_at',
    'trial_end_at',
    'user_id',
    'user_signup_date'
], axis=1)
val_df['churned_in_next_two_months'] = val_df['churned_in_next_two_months'].map({'t': True, 'f': False})
val_df['has_verified_email'] = val_df['has_verified_email'].map({'t': True, 'f': False})
val_df['has_notifications_enabled'] = val_df['has_notifications_enabled'].map({'t': True, 'f': False})
val_df['is_approved'] = val_df['is_approved'].map({'t': True, 'f': False})
val_df['is_suspended'] = val_df['is_suspended'].map({'t': True, 'f': False})
val_df = val_df.fillna(0)

In [113]:
X_val = val_df.drop(['subscription_id', 'churned_in_next_two_months'], axis=1)

In [114]:
Y_val = val_df['churned_in_next_two_months']

In [115]:
x_missing_cols = set( X_val.columns ) - set( X.columns )

In [116]:
for c in x_missing_cols:
    X[c] = 0

In [117]:
missing_cols = set( X.columns ) - set( X_val.columns )

In [118]:
for c in missing_cols:
    X_val[c] = 0

In [127]:
# clf = RandomForestClassifier(n_estimators=500, random_state=0)
clf = SVC()

In [128]:
clf.fit(X,Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [129]:
Y_val_pred = clf.predict(X_val)

In [130]:
roc_auc_score(Y_val, Y_val_pred)

0.5241544432446368